<a href="https://colab.research.google.com/github/Wisnih/NBA-players-value/blob/main/NBA_stats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fuzzywuzzy

# Importy i funkcje

In [ ]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import re
import string
from urllib.request import urlopen
import pandas as pd
import numpy as np
import time
from fuzzywuzzy import fuzz
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)

/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
class awards:

  def mvp(season):
    season = str(season)
    url = 'https://www.basketball-reference.com/awards/mvp.html'
    table_html = str(BeautifulSoup(urlopen(url), 'html.parser').findAll('table', id = 'mvp_NBA')[0])

    df = pd.read_html(table_html)[0]
    df.columns = df.columns.droplevel(0)
    df = df[['Season','Player']]
    df['Season'][0]

    for i in range(len(df)):
      df['Season'][i] = df['Season'][i][0:2] + df['Season'][i][5:7]

    mvp_lists = df.values.tolist()

    for data in mvp_lists:
      if season in data[0]:
        return data[1]

  def allstar(season):
    if season == 1999:
      print('No All Star game in 1999 season') # No all star game in 1999 season

    else:
      url = f'https://www.basketball-reference.com/allstar/NBA_{season}.html'
      name_html = BeautifulSoup(urlopen(url),'html.parser').findAll('a', string = re.compile('[a-z]'), href = re.compile('^/players/.+'), title= False)
      names = [names.text for names in name_html]
      names = list(set(names))

      return names


class get_data:

  def team_records(season):
    url = f'https://www.basketball-reference.com/leagues/NBA_{season}.html'
    table_html = BeautifulSoup(urlopen(url), 'html.parser').findAll('table', id = re.compile('advanced-team'))[0].findAll('a')
    table_html
    team_name = []
    team_abrv = []

    for html in table_html:
      abrv = html.get('href')
      pattern = re.compile(r'([A-Z]{3})')
      team_abrv.append(pattern.search(abrv).group())
      team_name.append(html.text)
    team_abrv = list(zip(team_name,team_abrv))

    table = BeautifulSoup(urlopen(url), 'html.parser').findAll('table', id = re.compile('advanced-team'))
    team_records = pd.read_html(str(table))[0]
    team_records = team_records.apply(pd.to_numeric, errors = 'coerce').fillna(team_records)
    team_records.columns = team_records.columns.droplevel(0)
    team_records = team_records.drop('Rk', axis=1) # drop Rk columns
    team_records = team_records.loc[:,['Team','W','L']] # only select team names and its winning and losing records
    team_records.Team = team_records.Team.str.replace('*','',regex=True) # remove team names asterisk

    team_names = [i[0] for i in team_abrv] # get list of team names
    team_abbrevation = [i[1] for i in team_abrv] # get list of team abbrevation
    for team,i in zip(team_records.Team,team_records.index.values): # the mapping process
      if team in team_names:
        idx = (team_names.index(team))
        team_records.Team[i] = team_abbrevation[idx]
      else:
        pass

    return team_records

  def single(season,stats,additional_data=False,salary=False):
    print('Loading',season,'data...')
    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_{stats}.html'
    table_html = BeautifulSoup(urlopen(url), 'html.parser').findAll('table')

    df = pd.read_html(str(table_html))[0]
    df = df.drop(df[df.Player == 'Player'].index) # drop row contains table header
    df = df.drop('Rk', axis=1) # drop Rk columns
    df.Player = df.Player.str.replace('*','',regex=True) # remove asterisk on player's name
    df.insert(0,'Season',season) # insert season column
    df = df.apply(pd.to_numeric, errors='coerce').fillna(df) # convert non string values to numeric

    # OPTIONAL
    # If you want to put all-star and mvp column to the player stats data frame, set additional_data=True in the argument. Note that it will slows the process.

    if additional_data:
    # Insert all-star column
      df['All_Star'] = 0
      if season == 1999:
        pass # No all star game in 1999 season
      else:
        all_star = awards.allstar(season)
        df.loc[df['Player'].isin(all_star), 'All_Star'] = 1  # change all star column to 1 if the player is in the all star list
      df[df.All_Star==1]

      # Insert MVP column
      df['MVP'] = 0
      mvp = awards.mvp(season)
      df.loc[df['Player'] == mvp, 'MVP'] = 1
      df[df.MVP == 1]

      # Insert team records
      team_records_list = get_data.team_records(season).values.tolist()
      df['team_win'] = 0
      df['team_lose'] = 0
      teams = [i[0] for i in team_records_list] # get list of team names
      win = [i[1] for i in team_records_list] # get list of team winning records
      lose = [i[2] for i in team_records_list] # get list of team losing records

      for team,i in zip(df.Tm,(df.Tm.index.values)): # there's a missing index so the iteration is going through the index values
        if team in teams:
          idx = (teams.index(team)) # get index in the team_records_list
          df.team_win[i]= win[idx] # add win records to df
          df.team_lose[i]= lose[idx] # add win records to df
        else:
          df.team_win[i] = 0
          df.team_lose[i] = 0

    if salary:
      # Get players' salary data from hoopshype.com
      def get_salary(season):
          season=str(int(season)-1)+'-'+str(int(season))
          url=f'https://hoopshype.com/salaries/players/{season}/'
          soup=BeautifulSoup(urlopen(url),'html.parser')
          salary=pd.read_html(str(soup))[0]
          salary=salary.iloc[:,[1,3]]
          salary=salary.rename(columns={salary.columns[1]:'Salary'})
          return salary

      stats=df
      salary=get_salary(season)
      salary_names=salary.Player.tolist()

      for salary_name in salary_names:
          stats.Player=stats.Player.apply(
              lambda stats_name: salary_name if fuzz.ratio(stats_name,salary_name)>=80
                      else stats_name)

      df=pd.merge(stats,salary,how='left',left_on='Player',right_on='Player')
    return df


  def multiple(start_year,end_year,stats):
    df = get_data.single(start_year,stats,additional_data=True, salary=True)

    begin_loop = time.time()

    while start_year < end_year:
      start_year = start_year + 1
      df_next = get_data.single(start_year,stats,additional_data=True, salary=True)

      lag = np.random.uniform(low=10,high=30)
      print(f'...wainting {round(lag, 1)} seconds')
      time.sleep(lag)

      df = pd.concat([df,df_next])

    print(f'Process completed! Total run time: {round((time.time()-begin_loop)/60, 2)} minutes')

    return df

# Pobranie danych

In [ ]:
#Pobieranie z NBA.com i Hoopshype.com
players_2013_2023 = get_data.multiple(2014 ,2024, 'advanced')
#players_2013 = get_data.single(2013, 'advanced', additional_data = False, salary = False)

Loading 2013 data...
Loading 2014 data...
...wainting 19.0 seconds
Loading 2015 data...
...wainting 18.0 seconds
Loading 2016 data...
...wainting 28.4 seconds
Loading 2017 data...
...wainting 20.8 seconds
Loading 2018 data...
...wainting 28.9 seconds
Loading 2019 data...
...wainting 24.6 seconds
Loading 2020 data...
...wainting 24.7 seconds
Loading 2021 data...
...wainting 14.0 seconds
Loading 2022 data...
...wainting 28.9 seconds
Loading 2023 data...
...wainting 24.0 seconds
Process completed! Total run time: 7.64 minutes


In [ ]:
players_2013_2024.sample(20)

,Season,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP,All_Star,MVP,team_win,team_lose,Salary
432,2020,Juwan Morgan,PF,22,UTA,21,134,10.1,0.648,0.308,0.154,8.5,15.1,11.9,6.2,0.4,1.2,17.8,11.0,NaN,0.2,0.1,0.3,0.106,NaN,-2.9,-0.3,-3.2,0.0,0,0,44,28,"$888,674"
154,2019,Mitchell Creek,SF,26,BRK,4,36,21.5,0.573,0.1,0.7,14.6,14.5,14.6,20.2,1.3,0.0,0.0,15.3,NaN,0.2,0.0,0.2,0.258,NaN,3.7,2.0,5.7,0.1,0,0,42,40,"$57,355"
4,2023,Santiago Aldama,PF,22,MEM,77,1682,13.9,0.591,0.507,0.274,5.4,18.0,11.7,7.6,1.3,2.6,9.3,16.0,NaN,2.1,2.4,4.6,0.13,NaN,-0.3,0.8,0.5,1.1,0,0,51,31,"$2,156,298"
566,2015,Jeff Teague,PG,26,ATL,73,2228,20.6,0.566,0.233,0.36,1.6,7.7,4.7,37.9,2.9,1.2,16.6,25.3,NaN,5.0,2.8,7.7,0.166,NaN,2.7,0.3,2.9,2.8,1,0,60,22,"$10,241,005"
106,2013,Daequan Cook,SG,25,TOT,49,441,5.8,0.398,0.545,0.072,1.8,14.1,7.9,6.7,0.9,1.2,8.0,19.2,NaN,-0.6,0.4,-0.2,-0.019,NaN,-3.5,-1.3,-4.8,-0.3,0,0,0,0,"$4,790,038"
609,2015,TJ Warren,SF,21,PHO,40,614,14.0,0.551,0.098,0.089,7.3,7.9,7.6,6.6,1.5,1.1,10.8,17.8,NaN,0.7,0.4,1.1,0.085,NaN,-0.6,-0.4,-0.9,0.2,0,0,39,43,"$2,500,239"
413,2015,Shabazz Muhammad,SG,22,MIN,38,866,19.9,0.556,0.128,0.365,8.1,12.4,10.2,9.6,1.1,0.6,7.1,25.3,NaN,2.1,-0.1,2.0,0.11,NaN,3.5,-2.7,0.7,0.6,0,0,16,66,"$2,524,356"
324,2013,Kyle Lowry,PG,26,TOR,68,2020,17.5,0.543,0.448,0.384,3.2,15.9,9.5,34.6,2.5,1.0,17.7,20.3,NaN,3.5,2.1,5.6,0.133,NaN,2.0,0.9,2.9,2.5,0,0,34,48,"$7,645,075"
74,2019,Corey Brewer,SF,32,TOT,31,492,11.6,0.532,0.358,0.35,5.4,10.7,8.0,10.2,3.0,1.2,11.8,13.6,NaN,0.4,0.6,1.0,0.095,NaN,-2.9,1.6,-1.3,0.1,0,0,0,0,"$3,099,564"
435,2020,Marcus Morris,PF,30,LAC,19,549,8.9,0.523,0.483,0.126,2.1,12.3,7.4,6.8,1.1,2.1,11.6,15.8,NaN,0.0,0.7,0.6,0.055,NaN,-2.5,0.3,-2.2,0.0,0,0,49,23,"$17,867,499"


In [ ]:
#Pobieranie z CSV
players_2024 = pd.read_csv('/content/drive/MyDrive/Licencjat/Per_game_stats/NBA_stats_2023_per_game (1).csv')

# Formatowanie danych

In [ ]:
def clean_currency(x):
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', ''))
    return(x)

def formating(players_stats):
  p = players_stats
  #Zmiana brakujących danych 'NaN' na 0
  p.fillna(0, inplace=True)

  #Zmiana w 'Salary' w float
  p['Salary'] = p['Salary'].apply(clean_currency).astype('float')
  return p


In [ ]:
p_2013_2024 = formating(players_2013_2024)
p_2013_2024 = p_2013_2024.drop(['Unnamed: 19','Unnamed: 24'], axis=1)
p_2013_2024

,Season,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,FTr,ORB%,DRB%,TRB%,AST%,STL%,BLK%,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,All_Star,MVP,team_win,team_lose,Salary
0,2013,Quincy Acy,PF,22,TOR,29,342,15.9,0.632,0.027,0.507,10.3,16.6,13.4,5.2,2.0,3.5,15.6,14.7,0.7,0.4,1.1,0.157,-1.0,1.0,0.1,0.2,0,0,34,48,884169.0
1,2013,Jeff Adrien,PF,26,CHA,52,713,13.4,0.493,0.012,0.595,10.6,21.2,15.7,8.3,1.3,3.1,13.1,15.6,0.5,0.4,1.0,0.064,-2.1,-0.9,-3.0,-0.2,0,0,21,61,868688.0
2,2013,Arron Afflalo,SF,27,ORL,64,2307,13.0,0.527,0.265,0.246,1.4,10.3,5.8,14.6,0.9,0.4,12.1,22.5,1.5,0.5,2.0,0.042,-1.1,-1.8,-2.9,-0.5,0,0,20,62,10054936.0
3,2013,Josh Akognon,PG,26,DAL,3,9,15.3,0.625,0.500,0.000,0.0,12.1,6.2,19.4,0.0,0.0,0.0,20.3,0.0,0.0,0.0,0.196,-3.1,0.1,-3.0,0.0,0,0,41,41,55560.0
4,2013,Cole Aldrich,C,24,TOT,45,388,11.1,0.563,0.000,0.250,8.7,26.7,17.7,3.4,0.7,4.6,20.6,12.7,0.1,0.4,0.6,0.070,-3.5,0.6,-2.9,-0.1,0,0,0,0,3251457.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
674,2023,Thaddeus Young,PF,34,TOR,54,795,14.1,0.573,0.172,0.131,9.4,14.6,11.8,12.9,3.4,0.6,16.7,13.5,0.7,1.1,1.8,0.109,-1.8,1.9,0.1,0.4,0,0,41,41,8237534.0
675,2023,Trae Young,PG,24,ATL,73,2541,22.0,0.573,0.331,0.460,2.4,7.0,4.7,42.5,1.5,0.3,15.2,32.6,5.3,1.4,6.7,0.126,5.3,-2.0,3.3,3.4,0,0,41,41,38197960.0
676,2023,Omer Yurtseven,C,24,MIA,9,83,16.7,0.675,0.259,0.222,10.9,21.9,16.2,3.9,1.2,2.5,11.9,18.0,0.2,0.1,0.3,0.159,-2.5,-1.5,-3.9,0.0,0,0,44,38,1804676.0
677,2023,Cody Zeller,C,30,MIA,15,217,16.4,0.659,0.034,0.593,13.0,21.8,17.3,7.2,0.7,1.9,15.8,18.1,0.4,0.3,0.7,0.147,-2.0,-0.7,-2.8,0.0,0,0,44,38,766010.0


In [ ]:
salary_data = p_2013_2024[['Season', 'Player', 'Pos', 'Salary']]
salary_data = salary_data.groupby(['Season']).mean().reset_index()
salary_data['Salary'] = salary_data['Salary'].apply(lambda x: round(x, 0))
salary_data

<ipython-input-61-c0b2dee1cc17>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  salary_data = salary_data.groupby(['Season']).mean().reset_index()


,Season,Salary
0,2013,5087980.0
1,2014,4917086.0
2,2015,5077257.0
3,2016,5780486.0
4,2017,6775150.0
5,2018,7047362.0
6,2019,7979823.0
7,2020,7349782.0
8,2021,8118268.0
9,2022,6745888.0


# Zapisywanie danych

In [ ]:
#players_2022.to_csv('NBA_stats_2022_per_game.csv')
p_2013_2023.to_excel('/content/drive/MyDrive/Licencjat/NBA_stats_2013_2023_advanced.xlsx', index=False)